In [1]:
import re
import requests
import time
import wikipedia
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# Clean data

In [46]:
out_f = '/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_wikipedia_tables_museums.csv'

In [47]:
df = pd.read_csv(out_f, index_col=0)
clean_df = df.copy()

/Users/klarnemann/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
n_rows, n_cols = df.shape
print(df.shape)

(29040, 31)


In [49]:
def combine_columns(df, cols):
    df = df.fillna('')
    for col in cols:
        try:
            values += df[col].astype(str)
        except:
            values = df[col].astype(str)
    col_entries = [np.array(df[col] != '').sum() for col in cols]
    value_entries = np.array(values != '').sum()
    if np.sum(col_entries) != value_entries:
        raise Exception('Columns do not align')
    return values

In [50]:
def replace_columns(df, cols, new_name, loc=None):
    values = combine_columns(df, cols)
    df = df.drop(cols, axis=1)
    if type(loc) == None:
        df.insert(loc, new_name, values)
    else:
        loc = len(df.columns) - 1
        df.insert(loc, new_name, values)
    return df

In [51]:
cols = ['Name', 'Museum Name', 'Museum name']
out_col = 'Name'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col, 0)

In [52]:
cols = ['Type', 'type', 'Area of study', 'Subject']
out_col = 'Category'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col)

In [53]:
cols = ['County', 'Parish', 'County or City', 'Borough']
out_col = 'County'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col)

In [54]:
cols = ['City', 'Town/City', 'Town/city', 'Location', 'Community']
out_col = 'City'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col)

In [55]:
cols = ['Region', 'Region[1]', 'Island']
out_col = 'Region'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col)

In [56]:
cols = ['State', 'Unnamed: 25']
out_col = 'State'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col)

In [57]:
cols = ['Website (if no article)', 'Website, Notes', 'Notes', 'Other information']
out_col = 'Notes'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col)

In [58]:
cols = ['Summary', 'Summary/Website', 'Website/Summary', 'Summary/Refs', 'Summary/refs', ]
out_col = 'Summary'
tmp = combine_columns(df, cols)
clean_df = replace_columns(clean_df, cols, out_col)

In [59]:
clean_df.columns

Index(['Name', 'Category', 'County', 'City', 'Region', 'State', 'Notes',
       'Summary', 'Article_content'],
      dtype='object')

In [63]:
clean_df = clean_df[['Name', 'Category', 'City', 'State', 'Article_content', 'Region', 'County', 'Summary', 'Notes']]

In [70]:
clean_df.to_csv('/Users/klarnemann/Documents/Insight/Insight_project/data/clean_scraped_wikipedia_tables_museums.csv')

In [43]:
29040 - clean_df.isnull().sum(axis=0)

Other information      554
Name                 29040
Category             29040
County               29040
City                 29040
Region               29040
Notes                29040
Summary              29040
State                29040
Article_content      23081
dtype: int64

In [45]:
vals = 29040 - np.array(clean_df == '').sum(axis=0)
for col, val in zip(clean_df.columns, vals):
    print(col,val)

Other information 29040
Name 29038
Category 29034
County 28900
City 29020
Region 25556
Notes 5018
Summary 23028
State 29040
Article_content 29040


# Get article content

In [5]:
title = 'Minnesota Lakes Maritime Museum'
page = wikipedia.page(title)
page.title

'Maritime museum'

In [12]:
for i in np.arange(n_rows):
    cols = ['Name', 'Museum Name', 'Museum name']
    if i % 1000 == 0:
        print(i)
        df.to_csv(out_f)
    for col in cols:
        title = df.loc[i, col]
        if title != '':
            try:
                df.loc[i, 'Article_content'] = wikipedia.page(title).content
                break
            except:
                break

0


/Users/klarnemann/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/klarnemann/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [13]:
df

,Name,State,Town/City,County,Region,Type,Summary,Borough,type,Location,...,"Website, Notes",Other information,Museum name,Unnamed: 25,City,Summary/Refs,Summary/refs,Summary/Website,County or City,Article_content
0,Alabama Administrative Office of Courts Museum...,Alabama,Montgomery,Montgomery,Central,Law,Exhibits of Alabama judicial history and impor...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama Artists Gallery,Alabama,Montgomery,Montgomery,Central,Art,Operated by the Alabama State Council on the A...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Fred Nall Hollis (born April 21, 1948) is an i..."
2,Alabama Constitution Village,Alabama,Huntsville,Madison,Northern,Living history,Open-air museum that reenacts life in 1819.[2],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Constitution Hall Park is a historical ope...
3,Alabama Department of Archives and History,Alabama,Montgomery,Montgomery,Central,History,"Official archives for the state of Alabama, fe...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Alabama Department of Archives and History...
4,Alabama Governor's Mansion,Alabama,Montgomery,Montgomery,Central,Historic house,Official residence of Alabama's Governor. Open...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Alabama Governor's Mansion is the official...
5,Alabama Historical Radio Museum,Alabama,Birmingham,Jefferson,Central,Radio,Exhibits include antique radios. Operated by t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The list of museums in Alabama contains museum...
6,Alabama Jazz Hall of Fame,Alabama,Birmingham,Jefferson,Central,Music,Hall of Fame honors jazz artists with ties to ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Alabama Jazz Hall of Fame (AJHF) was found...
7,Alabama Men's Hall of Fame,Alabama,Birmingham,Jefferson,Central,History,Hall of Fame for men native to or closely iden...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Robert Sylvester Munger (July 24, 1854 – April..."
8,Alabama Mining Museum,Alabama,Dora,Walker,Central,Industry,Exhibits include a 1900s train and mining cars...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The list of museums in Alabama contains museum...
9,Alabama Museum of Health Sciences,Alabama,Birmingham,Jefferson,Central,Medical,Displays illustrating the health sciences in t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Alabama Museum of the Health Sciences was ...


# Get tables for all states

In [ ]:
def get_table_from_wikipedia(state, url, skiprows=0):
    t0 = time.time()
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    response_time = time.time() - t0
    table = soup.find("table",{"class":"wikitable sortable"})
    df = pd.read_html(str(table), skiprows=skiprows)[0]
    n_rows, _ = df.shape
    df.insert(1, 'State', [state]*n_rows, True) 
    return df, response_time

In [ ]:
for state, state_url in zip(states, state_urls):
    if state == 'Georgia':
        state_url = 'https://en.wikipedia.org/wiki/List_of_museums_in_Georgia_(U.S._state)'
    elif state == 'New York':
        state_url = 'https://en.wikipedia.org/wiki/List_of_museums_in_New_York_(state)'
    elif state == 'Texas':
        for texas_url in texas_urls:
            try:
                df, response_time = get_table_from_wikipedia(state, texas_url, 0)
                out_df = out_df.append(df, sort=False, ignore_index=True)
            except:
                df, response_time = get_table_from_wikipedia(state, texas_url, 0)
                df.columns = df.columns.droplevel()
                out_df = out_df.append(df, sort=False, ignore_index=True)
            time.sleep(10*response_time)
        continue
    df, response_time = get_table_from_wikipedia(state, state_url)
    out_df = out_df.append(df, sort=False, ignore_index=True)
    time.sleep(10*response_time)

In [ ]:
states = np.loadtxt('/Users/klarnemann/Documents/Insight/Insight_project/data/states.txt', delimiter='\t', dtype='str')
states = list(states)

In [ ]:
state_urls = ['https://en.wikipedia.org/wiki/List_of_museums_in_%s' % (re.sub(' ','_', s)) for s in states]

In [ ]:
texas_urls = ['https://en.wikipedia.org/wiki/List_of_museums_in_Central_Texas',\
              'https://en.wikipedia.org/wiki/List_of_museums_in_East_Texas',\
              'https://en.wikipedia.org/wiki/List_of_museums_in_North_Texas', \
              'https://en.wikipedia.org/wiki/List_of_museums_in_the_Texas_Gulf_Coast',\
              'https://en.wikipedia.org/wiki/List_of_museums_in_the_Texas_Panhandle']

In [ ]:
out_df.columns

In [ ]:
df, _ = get_table_from_wikipedia(states[0], state_urls[0])
df

In [ ]:
for state in states: 
    print(state, np.sum(out_df['State'] == state))

In [ ]:
out_df.columns

In [ ]:
out_df = out_df.dropna(how='all', axis=0)
out_df = out_df.dropna(how='all', axis=1)
out_df = out_df.reset_index()
out_df = out_df.drop(['index', 'level_0', 'Image'], axis=1)
out_df

In [ ]:
out_f = '/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_wikipedia_tables_museums.csv'
out_df.to_csv(out_f)